In [1]:
!pip install plotly==4.14.1
# !pip install dash==1.18.1
# !pip install jupyter-dash
# !pip install dash-bootstrap-components
!pip install pyngrok

     |████████████████████████████████| 13.2MB 4.0MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1
  Created wheel for pyngrok: filename=pyngrok-5.0.1-cp36-none-any.whl size=18822 sha256=5e5c7c3a41d40ca8836081d0c583b1d285d287edb78fc252fa40f5e80422b797
  Stored in directory: /root/.cache/pip/wheels/94/01/05/d39efb8f6b40a411354b4168ca9dda99e6f8d586e458e97551
Successfully built pyngrok


In [2]:
import pandas as pd
import numpy as np
import datetime
# import dash
# import dash_table as dt
# import dash_core_components as dcc
# import dash_bootstrap_components as dbc
# import dash_html_components as html
import plotly
import plotly.express as px
import plotly.graph_objects as go
# from jupyter_dash import JupyterDash
# from dash.dependencies import Input, Output

### **Question:**
* What is the overall sales trend?

* Which are the Top 10 products by sales?

* Which are the Most Selling Products?

* Which is the most preferred Ship Mode?

* Which are the Most Profitable Category and Sub-Category?

In [40]:
home = '/content/drive/MyDrive/ai/machine_learning/superstore-sales-analysis/'
df = pd.read_excel(f'{home}superstore_sales.xlsx')
df.head(10)

,order_id,order_date,ship_date,ship_mode,customer_name,segment,state,country,market,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,year
0,AG-2011-2040,2011-01-01,2011-01-06,Standard Class,Toby Braunhardt,Consumer,Constantine,Algeria,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.300,2,0.0,106.140,35.46,Medium,2011
1,IN-2011-47883,2011-01-01,2011-01-08,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.366,3,0.1,36.036,9.72,Medium,2011
2,HU-2011-1220,2011-01-01,2011-01-05,Second Class,Annie Thurman,Consumer,Budapest,Hungary,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.120,4,0.0,29.640,8.17,High,2011
3,IT-2011-3647632,2011-01-01,2011-01-05,Second Class,Eugene Moren,Home Office,Stockholm,Sweden,EU,North,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.865,3,0.5,-26.055,4.82,High,2011
4,IN-2011-47883,2011-01-01,2011-01-08,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.670,5,0.1,37.770,4.70,Medium,2011
5,IN-2011-47883,2011-01-01,2011-01-08,Standard Class,Joseph Holt,Consumer,New South Wales,Australia,APAC,Oceania,OFF-PA-10001968,Office Supplies,Paper,"Eaton Computer Printout Paper, 8.5 x 11",55.242,2,0.1,15.342,1.80,Medium,2011
6,CA-2011-1510,2011-01-02,2011-01-06,Standard Class,Magdelene Morse,Consumer,Ontario,Canada,Canada,Canada,TEC-OKI-10002750,Technology,Machines,"Okidata Inkjet, Wireless",314.220,1,0.0,3.120,24.10,Medium,2011
7,IN-2011-79397,2011-01-03,2011-01-03,Same Day,Kean Nguyen,Corporate,New South Wales,Australia,APAC,Oceania,OFF-AP-10000304,Office Supplies,Appliances,"Hoover Microwave, White",276.102,1,0.1,110.412,125.32,Critical,2011
8,ID-2011-80230,2011-01-03,2011-01-09,Standard Class,Ken Lonsdale,Consumer,Auckland,New Zealand,APAC,Oceania,TEC-CO-10004182,Technology,Copiers,"Hewlett Wireless Fax, Laser",912.456,4,0.4,-319.464,107.10,Low,2011
9,IZ-2011-4680,2011-01-03,2011-01-07,Standard Class,Lindsay Williams,Corporate,Ninawa,Iraq,EMEA,EMEA,FUR-NOV-10002791,Furniture,Chairs,"Novimex Swivel Stool, Set of Two",666.840,4,0.0,253.320,81.26,High,2011


In [4]:
# df.info()

In [5]:
# df.isna().sum()

In [6]:
# df.describe().round()

### **WHAT IS THE OVERALL SALES TREND?**

In [87]:
df['month_year'] = df['order_date'].apply(lambda x: x.strftime('%Y-%m'))
overall_sales = df.groupby(['month_year', 'year'])['sales'].sum().reset_index()
overall_trend = overall_sales.sort_values(by=['sales'], ascending=False)

In [98]:
fig = px.bar(overall_trend, x='month_year', y='sales', color='year', barmode='group',
             height=350, labels={'month_year':'Sales per-Months (Year)', 'sales':'Number of Sales'})
fig.show()

### **WHICH ARE THE TOP 10 PRODUCTS BY SALES?**



In [58]:
def get_word(w):
  return w.split(', ')[0]

df['product'] = df['product_name'].apply(lambda x: f'{get_word(x)}')
df.drop(['product_name'], inplace=True, axis=1)

In [220]:
prod_sales = df.groupby(['product', 'year'])['sales'].sum().reset_index()
prod_sales10 = prod_sales.sort_values(by=['sales'], ascending=False).iloc[0:5]

In [221]:
fig = px.bar(prod_sales10, x='product', y='sales', color='year', barmode='group',
             height=350, labels={'product':'Top 10 Sales by Products', 'sales':'Number of Sales', 'year':'Range by Year'})
fig.show()

### **WHICH ARE THE MOST SELLING PRODUCTS?**

In [233]:
most_sales = df.groupby(['product', 'year'])['quantity'].sum().reset_index()
most_sales10 = most_sales.sort_values(by=['quantity'], ascending=False).iloc[0:5]

In [234]:
fig = px.bar(most_sales10, x='product', y='quantity', color='year', barmode='group',
             height=350, labels={'year':'Year', 'quantity':'Quantity', 'product':'Most Selling Products'})
fig.show()

### **WHAT IS THE MOST PREFERRED SHIP MODE?**

In [191]:
fig = px.histogram(df, x='ship_mode', color='ship_mode', height=350, labels={'ship_mode':'Shipping Mode'})
fig.show()

In [ ]:
'''def category():
  df_cat = df.groupby('category')['profit'].sum().reset_index()
  df_cat.sort_values(by=['profit'], ascending=False)
  return html.Div([
    html.P('Most Profitable Category', 
        style={'text-align':'center', 'color':'white', 
                'padding-top':'20px', 'padding-bottom':'20px'}),

    dcc.Graph(
        figure = px.pie(df_cat, values='profit', names='category', 
                labels={'profit':'Profit'}
                ).update_layout({'plot_bgcolor': '#073642', 'paper_bgcolor':'#073642'}, 
                                font={'color':'gray'}
                ), config={'displayModeBar':'hover'}, style={'height':'350px'})
  ])
  
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SOLAR],
                meta_tags=[{'name':'viewport',
                'content':'width=device-width, initial-scale=1.0'}])

app.layout = dbc.Container([

    dbc.Row(
        dbc.Col(html.H2('Super Store Dashboard', 
                    style={'color':'white', 'padding-top':'20px', 'padding-bottom':'30px'}, 
                    className='text-center lg-4'), width=12)),

    dbc.Row([
        dbc.Col([
            html.P('Most Profitable Sub Category by Region', 
                    style={'color':'white', 'padding-top':'20px', 'padding-bottom':'10px'},
                   className='text-center lg-6'
            ),

            dbc.RadioItems(
                    id='radio_sub', value='Sub Category', 
                    style={'color':'white', 'text-align':'center'},
                    options=[
                        {'label':'Sub Category', 'value':'Sub Category'},
                        {'label':'Region', 'value':'Region'},
                    ], inline=True
            ),

            dcc.Graph(id='sub_fig', config={'displayModeBar':'hover'}, style={'height':'350px'})
        ], width=6, id='col-con'),
             
        dbc.Col([category()], width=6, id='col-con'),
    ]),

    dbc.Row([
        dbc.Col([], width=6, id='col-con'),
    ]),
])

# Define callback to update graph
@app.callback(Output('sub_fig', 'figure'),
              Input('radio_sub', 'value'))

def update_graph(radio_sub):
  most_profit_sub = df.groupby('sub_category')[['profit', 'product_name']].sum().reset_index()
  sub_top_profit= most_profit_sub.sort_values(by=['profit'], ascending=False).iloc[0:10]
  region = df.groupby(['year', 'segment', 'region'])['profit'].sum().reset_index()
  region_top = region.sort_values(by=['profit'], ascending=False).iloc[0:10]

  if radio_sub == 'Sub Category':
      fig = px.bar(sub_top_profit, x='sub_category', y='profit', 
                  color='sub_category', labels={'sub_category':'Sub Category', 'profit':'Profit'})
      fig.update_layout({'plot_bgcolor': '#073642', 'paper_bgcolor':'#073642'}, 
                        xaxis=dict(showgrid=True), yaxis=dict(showgrid=True),
                        font={'color':'gray'})
      return fig

  elif radio_sub == 'Region':
      fig = px.bar(region_top, x='region', y='profit', 
                  color='region', labels={'region_top':'Region', 'profit':'Profit'})
      fig.update_layout({'plot_bgcolor': '#073642', 'paper_bgcolor':'#073642'}, 
                        xaxis=dict(showgrid=True), yaxis=dict(showgrid=True), 
                        font={'color':'gray'})
      return fig


if __name__ == '__main__':
  app.run_server(mode='external', port=7339)'''

In [ ]:
# from pyngrok import ngrok
# public_url = ngrok.connect(7337, 'http')

In [ ]:
# public_url

In [ ]:
# ngrok.kill()